In [1]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np

2024-06-22 08:39:24.870069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 08:39:24.870186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 08:39:25.041498: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!pip install tensorflow
!pip install tf2onnx
!pip install onnx

import tf2onnx
import onnx
#from tensorflow import keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 6.6 MB/s eta 0:00:00a 0:00:01


In [25]:
from joblib import load
import keras
model = keras.models.load_model('/kaggle/input/new-hrw-d/diagnostic_2/12_led/lstm_model_d.keras')

In [26]:
list_corr_rhythm = pd.read_csv("/kaggle/input/new-hrw-d/diagnostic_2/list_corr_diagnostic.csv", index_col=0)


In [27]:
len(list(list_corr_rhythm["0"][:-1]))

40

In [28]:
input_signature = [tf.TensorSpec((1, 40), tf.float32, name='x')]
model.output_names = ["output_names"]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)
onnx.save(onnx_model, "lstm_model_d.onnx")

In [29]:
onnx_model2 = onnx.load("./lstm_model_d.onnx")
#onnx.checker.check_model(onnx_model)

In [8]:
path = '/kaggle/input/electrocardiogram-database-for-arrhythmia-study/ecg_df_diagnostic.csv'

ecg_df_HTV = pd.read_csv(path, index_col=0 )

In [30]:
X_valid =ecg_df_HTV[list(list_corr_rhythm["0"][:-1])]

In [10]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 39.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.3 MB/s eta 0:00:00


In [31]:
d = X_valid.to_numpy().astype(np.float32)

In [38]:
import torch
import onnx

# Загружаем ONNX модель
#onnx_model = onnx.load("./lstm_model_r.onnx")

# Создаем экземпляр onnxruntime для запуска ONNX модели
import onnxruntime as ort
ort_session = ort.InferenceSession("./lstm_model_r.onnx")

# Создаем входной тензор PyTorch
#input_tensor = torch.randn(10, 4)  # Пример: batch size 10, 4 признака
        

# Запускаем ONNX модель с помощью onnxruntime
output = ort_session.run(["output_names"], {'x': d[:1]})[0]  

# Преобразуем выходные данные в тензор PyTorch (если необходимо)
output_tensor = torch.from_numpy(output)

print(output_tensor)

tensor([[0.6678, 0.9953, 0.0948, 0.1890, 0.7323, 0.3242]])


In [36]:
pd.DataFrame(output_tensor).T.idxmax()

0    1
dtype: int64

## Rename uniques

In [27]:
!pip install wget
import wget
!pip install googletrans

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15715 sha256=3cb0a28445937998d24fa01b95c71678957f9faa54b8a5314e8768cca509a566
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Success

In [5]:
import wfdb

In [7]:
import wget
import os


url = "https://coronavirus.data.gov.uk/downloads/csv/coronavirus-cases_latest.csv"#

wget.download("https://physionet.org/files/ptb-xl/1.0.3/scp_statements.csv")

'scp_statements.csv'

In [9]:
scp_statements = pd.read_csv("./scp_statements.csv")
uniques_diagnostic = pd.read_csv("/kaggle/input/new-hrw-d/diagnostic_2/uniques_diagnostic.csv", index_col=0)
uniques_rhythm = pd.read_csv("/kaggle/input/new-hrw-d/rhythm_2/uniques_rhythm.csv", index_col=0)

In [36]:
!pip install -U deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00


In [55]:
def f_0(uniques,desc):
    l = []
    r = []
    for i in uniques["0"]:
        list_i = i.split(",")
        name = ""
        for j in list_i:
            k = list(scp_statements[scp_statements["Unnamed: 0"] == j][desc])[0]
            k = k.capitalize()
            name = name+k+","
        l.append(name[:-1])
        r.append(GoogleTranslator(source='auto', target='ru').translate(name[:-1]))
        
    return l,r

In [46]:
uniques_rhythm["en"],uniques_rhythm["ru"]  = f_0(uniques_rhythm,"description")

In [57]:
uniques_diagnostic["en"],uniques_diagnostic["ru"]  = f_0(uniques_diagnostic,"description")

In [47]:
uniques_rhythm.to_csv("uniques_rhythm.csv")

In [58]:
uniques_diagnostic.to_csv("uniques_diagnostic.csv")